**stage3_model_eval**. This notebook evaluates the trained model

**Edit**<br/>

**TODO**<br/>

# Import packages and get authenticated

In [1]:
# from google.colab import drive
# drive.mount('drive')

In [7]:
import numpy as np
import copy
import pandas as pd
pd.set_option("display.precision", 5)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm_notebook as tqdm
from IPython.display import display
import os
import sys
sys.path.append('/content/drive/My Drive/中研院/repo/')
# sys.path.append('~/project_FDDAT/repo/')
sys.path.append('../') # add this line so Data and data are visible in this file
from os.path import expanduser
home = expanduser("~")

from falldetect.utilities import *
from falldetect.models import *
from falldetect.dataset_util import *
from falldetect.training_util import *
from falldetect.eval_util import *

import time
import datetime
from datetime import datetime
import json
import argparse

# Plotting
# checklist 1: comment inline, uncomment Agg
%matplotlib inline
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
matplotlib.rc( 'savefig', facecolor = 'white' )

from sklearn.decomposition import PCA

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Get user inputs
In ipython notebook, these are hardcoded. In production python code, use parsers to provide these inputs

In [8]:
parser = argparse.ArgumentParser(description='FD_DAT')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
# parser.add_argument('--extractor_type', metavar='extractor_type', help='extractor_type',
#                     default='CNN')
# parser.add_argument('--num_epochs', type=int, metavar='num_epochs', help='number of epochs',
#                     default='5')
# parser.add_argument('--CV_n', type=int, metavar='CV_n', help='CV folds',
#                     default='2')
# parser.add_argument('--rep_n', type=int, metavar='rep_n', help='number of repitition',
#                     default='5')
parser.add_argument('--tasks_list', metavar='tasks_list', help='a list of all tasks',
                    default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')
parser.add_argument('--src_names', metavar='src_names', help='a list of src_names',
                    default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')
parser.add_argument('--tgt_names', metavar='tgt_names', help='a list of tgt_names',
                    default='UMAFall_waist_UPFall_belt UPFall_wrist_UMAFall_ankle')



# split_mode = 'LOO'
# split_mode = '5fold'


# checklist 2: comment first line, uncomment second line seizures_FN
args = parser.parse_args(['--input_folder', '../../data_mic/stage2_modeloutput_WithoutNormal_18hz_5fold_UPFall_UMAFall_cross-config', 
                          '--output_folder', '../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF',
                          '--tasks_list', 'UPFall_neck-UMAFall_chest UPFall_wrist-UMAFall_wrist UPFall_belt-UMAFall_waist UPFall_rightpocket-UMAFall_leg UPFall_ankle-UMAFall_ankle',
                          '--src_names',  'UPFall_neck UPFall_wrist UPFall_belt UPFall_rightpocket UPFall_ankle',
                          '--tgt_names',  'UMAFall_chest UMAFall_wrist UMAFall_waist UMAFall_leg UMAFall_ankle',

                         ])
# args = parser.parse_args()

In [9]:
home_dir = home+'/project_FDDAT/'
input_folder = args.input_folder
output_folder = args.output_folder

tasks_list = []
for item in args.tasks_list.split(' '):
    tasks_list.append((item.split('-')[0], item.split('-')[1]))
    
src_domains = args.src_names.split(' ')
tgt_domains = args.tgt_names.split(' ')

inputdir = input_folder + '/'
outputdir = output_folder + '/'


In [10]:
df_metric_keys = ['df_acc', 'df_sensitivity', 'df_precision', 'df_F1']
df_sample = pd.DataFrame(float('NaN'), columns=tgt_domains, index=src_domains)
df_sample = df_sample.loc[:,~df_sample.columns.duplicated()]
df_sample = df_sample.loc[~df_sample.index.duplicated(keep='first')]

df_dict_cross_config = dict( zip(df_metric_keys,[df_sample.copy(), df_sample.copy(), df_sample.copy(), df_sample.copy()]))

                                  
training_setting_list = ['source', 'DANN', 'target', 'improvement']
finetuned_results = dict( zip(training_setting_list,[copy.deepcopy(df_dict_cross_config), copy.deepcopy(df_dict_cross_config), copy.deepcopy(df_dict_cross_config), copy.deepcopy(df_dict_cross_config), ]))

# df_task_list = []
for task_item in tasks_list:
    for training_setting in training_setting_list:
        for metric_name in df_metric_keys:
            if training_setting == 'improvement':
                finetuned_results[training_setting][metric_name].at[src_name, tgt_name] = finetuned_results['DANN'][metric_name].at[src_name, tgt_name] - finetuned_results['source'][metric_name].at[src_name, tgt_name] 
                continue
            (src_name, tgt_name) = task_item
            df_task_inputdir = inputdir+src_name+'_'+tgt_name+'/repetitive_results/df_performance_table_agg_rep_{}.csv'.format(metric_name.split('_')[1])
            if os.path.isfile(df_task_inputdir):
                df_task = pd.read_csv(df_task_inputdir, index_col=0)[['rep_avg']].copy()
            else:
                continue
            df_task.rename(columns={'rep_avg':src_name+'_'+tgt_name}, inplace=True)

            finetuned_results[training_setting][metric_name].at[src_name, tgt_name] = float(df_task.copy().iloc[:, 0][training_setting].split('±')[0])

for training_setting in finetuned_results.keys():
    for metric_name in finetuned_results[training_setting].keys():
        print('training_setting: {}, metric_name: {}'.format(training_setting, metric_name))
        df_outputdir = '{}{}/'.format(outputdir, metric_name.split('_')[1])
        if not os.path.exists(df_outputdir):
            os.makedirs(df_outputdir)
        print('will export data to', df_outputdir)

        finetuned_results[training_setting][metric_name].round(5).to_csv(df_outputdir+'df_performance_table_agg_{}.csv'.format(training_setting), encoding='utf-8')
        display(finetuned_results[training_setting][metric_name])

training_setting: source, metric_name: df_acc
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/acc/


,UMAFall_chest,UMAFall_wrist,UMAFall_waist,UMAFall_leg,UMAFall_ankle
UPFall_neck,0.984,NaN,NaN,NaN,NaN
UPFall_wrist,NaN,0.968,NaN,NaN,NaN
UPFall_belt,NaN,NaN,0.983,NaN,NaN
UPFall_rightpocket,NaN,NaN,NaN,0.656,NaN
UPFall_ankle,NaN,NaN,NaN,NaN,0.941


training_setting: source, metric_name: df_sensitivity
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/sensitivity/


,UMAFall_chest,UMAFall_wrist,UMAFall_waist,UMAFall_leg,UMAFall_ankle
UPFall_neck,0.965,NaN,NaN,NaN,NaN
UPFall_wrist,NaN,0.993,NaN,NaN,NaN
UPFall_belt,NaN,NaN,0.967,NaN,NaN
UPFall_rightpocket,NaN,NaN,NaN,0.173,NaN
UPFall_ankle,NaN,NaN,NaN,NaN,0.947


training_setting: source, metric_name: df_precision
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/precision/


...

,UMAFall_chest,UMAFall_wrist,UMAFall_waist,UMAFall_leg,UMAFall_ankle
UPFall_neck,0.012,NaN,NaN,NaN,NaN
UPFall_wrist,NaN,0.0,NaN,NaN,NaN
UPFall_belt,NaN,NaN,0.013,NaN,NaN
UPFall_rightpocket,NaN,NaN,NaN,0.537,NaN
UPFall_ankle,NaN,NaN,NaN,NaN,-0.065


training_setting: improvement, metric_name: df_precision
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/precision/


,UMAFall_chest,UMAFall_wrist,UMAFall_waist,UMAFall_leg,UMAFall_ankle
UPFall_neck,-0.013,NaN,NaN,NaN,NaN
UPFall_wrist,NaN,0.018,NaN,NaN,NaN
UPFall_belt,NaN,NaN,-0.012,NaN,NaN
UPFall_rightpocket,NaN,NaN,NaN,-0.016,NaN
UPFall_ankle,NaN,NaN,NaN,NaN,0.019


training_setting: improvement, metric_name: df_F1
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/F1/


,UMAFall_chest,UMAFall_wrist,UMAFall_waist,UMAFall_leg,UMAFall_ankle
UPFall_neck,0.0,NaN,NaN,NaN,NaN
UPFall_wrist,NaN,0.01,NaN,NaN,NaN
UPFall_belt,NaN,NaN,0.001,NaN,NaN
UPFall_rightpocket,NaN,NaN,NaN,0.502,NaN
UPFall_ankle,NaN,NaN,NaN,NaN,-0.022


In [11]:
df_task_list = dict( zip(df_metric_keys,[[], [], [], []]))

for task_item in tasks_list:
    for metric_name in df_metric_keys:
        (src_name, tgt_name) = task_item
        df_task_inputdir = inputdir+src_name+'_'+tgt_name+'/repetitive_results/df_performance_table_agg_rep_{}.csv'.format(metric_name.split('_')[1])
        if os.path.isfile(df_task_inputdir):
            df_task = pd.read_csv(df_task_inputdir, index_col=0)[['rep_avg']].copy()
        else:
            continue
        df_task.rename(columns={'rep_avg':src_name+'_'+tgt_name}, inplace=True)
        df_task.loc['improvement'] = float(df_task.loc['DANN'].values[0].split('±')[0])-float(df_task.loc['source'].values[0].split('±')[0])
        df_task_list[metric_name].append(df_task)
    
    
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(outputdir+'{}_allmetrics.xlsx'.format(output_folder.split('/')[-1]), engine='xlsxwriter')

for metric_name in df_metric_keys:
    print(metric_name)
    if not os.path.exists(outputdir):
        os.makedirs(outputdir)
    print('will export data to', outputdir)
    
    df_task_list[metric_name] = pd.concat(df_task_list[metric_name], axis=1)
    df_task_copy = df_task_list[metric_name].copy()
    df_task_list[metric_name].at['source', 'average'] = df_task_copy.loc['source'].apply(get_mean).mean()
    df_task_list[metric_name].at['DANN', 'average'] = df_task_copy.loc['DANN'].apply(get_mean).mean()
    df_task_list[metric_name].at['target', 'average'] = df_task_copy.loc['target'].apply(get_mean).mean()
    df_task_list[metric_name].at['domain', 'average'] = df_task_copy.loc['domain'].apply(get_mean).mean()
    df_task_list[metric_name].at['improvement', 'average'] = df_task_copy.loc['improvement'].mean()
    df_task_list[metric_name] = df_task_list[metric_name].reindex(['channel_n', 'batch_size', 'learning_rate', 'time_elapsed', 'num_params', \
                  'source', 'DANN', 'target', 'domain', 'improvement'])
    df_task_list[metric_name].to_csv(outputdir+'df_performance_table_agg_{}.csv'.format(metric_name.split('_')[1]), encoding='utf-8')
    df_task_list[metric_name].to_excel(writer, sheet_name=metric_name.split('_')[1])
    display(df_task_list[metric_name])

writer.save()

df_acc
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/


,UPFall_neck_UMAFall_chest,UPFall_wrist_UMAFall_wrist,UPFall_belt_UMAFall_waist,UPFall_rightpocket_UMAFall_leg,UPFall_ankle_UMAFall_ankle,average
channel_n,16.0,16.0,16.0,16.0,4.0,NaN
batch_size,4.0,16.0,4.0,4.0,4.0,NaN
learning_rate,0.01,0.001,0.01,0.01,0.001,NaN
time_elapsed,322.63351678848267,142.87455582618713,410.03790683746337,434.01085929870607,420.43386487960817,NaN
num_params,4132.0,4132.0,4132.0,4132.0,748.0,NaN
source,0.984±0.005,0.968±0.003,0.983±0.008,0.656±0.016,0.941±0.013,0.9064
DANN,0.983±0.007,0.976±0.004,0.983±0.005,0.872±0.045,0.927±0.007,0.9482
target,0.984±0.003,0.980±0.005,0.989±0.005,0.953±0.021,0.941±0.014,0.9694
domain,0.561±0.041,0.581±0.017,0.521±0.058,0.514±0.039,0.546±0.026,0.5446
improvement,-0.001,0.008,0,0.216,-0.014,0.0418


df_sensitivity
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/


,UPFall_neck_UMAFall_chest,UPFall_wrist_UMAFall_wrist,UPFall_belt_UMAFall_waist,UPFall_rightpocket_UMAFall_leg,UPFall_ankle_UMAFall_ankle,average
channel_n,16.0,16.0,16.0,16.0,4.0,NaN
batch_size,4.0,16.0,4.0,4.0,4.0,NaN
learning_rate,0.01,0.001,0.01,0.01,0.001,NaN
time_elapsed,322.63351678848267,142.87455582618713,410.03790683746337,434.01085929870607,420.43386487960817,NaN
num_params,4132.0,4132.0,4132.0,4132.0,748.0,NaN
source,0.965±0.014,0.993±0.000,0.967±0.020,0.173±0.036,0.947±0.017,0.8090
DANN,0.977±0.007,0.993±0.000,0.980±0.014,0.710±0.116,0.882±0.020,0.9084
target,0.970±0.005,0.970±0.009,0.974±0.007,0.925±0.044,0.850±0.043,0.9378
domain,0.561±0.041,0.581±0.017,0.521±0.058,0.514±0.039,0.546±0.026,0.5446
improvement,0.012,0,0.013,0.537,-0.065,0.0994


df_precision
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/


,UPFall_neck_UMAFall_chest,UPFall_wrist_UMAFall_wrist,UPFall_belt_UMAFall_waist,UPFall_rightpocket_UMAFall_leg,UPFall_ankle_UMAFall_ankle,average
channel_n,16.0,16.0,16.0,16.0,4.0,NaN
batch_size,4.0,16.0,4.0,4.0,4.0,NaN
learning_rate,0.01,0.001,0.01,0.01,0.001,NaN
time_elapsed,322.63351678848267,142.87455582618713,410.03790683746337,434.01085929870607,420.43386487960817,NaN
num_params,4132.0,4132.0,4132.0,4132.0,748.0,NaN
source,0.994±0.002,0.934±0.006,0.993±0.004,0.993±0.006,0.875±0.017,0.9578
DANN,0.981±0.021,0.952±0.009,0.981±0.013,0.977±0.018,0.894±0.008,0.9570
target,0.989±0.006,0.982±0.004,0.997±0.004,0.964±0.016,0.970±0.018,0.9804
domain,0.561±0.041,0.581±0.017,0.521±0.058,0.514±0.039,0.546±0.026,0.5446
improvement,-0.013,0.018,-0.012,-0.016,0.019,-0.0008


df_F1
will export data to ../../data_mic/stage3/UMAFall_UPFall_cross_config_HPF/


,UPFall_neck_UMAFall_chest,UPFall_wrist_UMAFall_wrist,UPFall_belt_UMAFall_waist,UPFall_rightpocket_UMAFall_leg,UPFall_ankle_UMAFall_ankle,average
channel_n,16.0,16.0,16.0,16.0,4.0,NaN
batch_size,4.0,16.0,4.0,4.0,4.0,NaN
learning_rate,0.01,0.001,0.01,0.01,0.001,NaN
time_elapsed,322.63351678848267,142.87455582618713,410.03790683746337,434.01085929870607,420.43386487960817,NaN
num_params,4132.0,4132.0,4132.0,4132.0,748.0,NaN
source,0.978±0.008,0.962±0.003,0.979±0.010,0.305±0.054,0.905±0.019,0.8258
DANN,0.978±0.008,0.972±0.004,0.980±0.006,0.807±0.059,0.883±0.013,0.9240
target,0.979±0.003,0.975±0.006,0.985±0.006,0.940±0.030,0.902±0.030,0.9562
domain,0.561±0.041,0.581±0.017,0.521±0.058,0.514±0.039,0.546±0.026,0.5446
improvement,0,0.01,0.001,0.502,-0.022,0.0982


In [12]:
display(df_task_list[metric_name])


,UPFall_neck_UMAFall_chest,UPFall_wrist_UMAFall_wrist,UPFall_belt_UMAFall_waist,UPFall_rightpocket_UMAFall_leg,UPFall_ankle_UMAFall_ankle,average
channel_n,16.0,16.0,16.0,16.0,4.0,NaN
batch_size,4.0,16.0,4.0,4.0,4.0,NaN
learning_rate,0.01,0.001,0.01,0.01,0.001,NaN
time_elapsed,322.63351678848267,142.87455582618713,410.03790683746337,434.01085929870607,420.43386487960817,NaN
num_params,4132.0,4132.0,4132.0,4132.0,748.0,NaN
source,0.978±0.008,0.962±0.003,0.979±0.010,0.305±0.054,0.905±0.019,0.8258
DANN,0.978±0.008,0.972±0.004,0.980±0.006,0.807±0.059,0.883±0.013,0.9240
target,0.979±0.003,0.975±0.006,0.985±0.006,0.940±0.030,0.902±0.030,0.9562
domain,0.561±0.041,0.581±0.017,0.521±0.058,0.514±0.039,0.546±0.026,0.5446
improvement,0,0.01,0.001,0.502,-0.022,0.0982
